## Basic Dask

What is an efficient way to process very large data? What if we do not have enough RAM to perform the calculations? Dask to the rescue! Dask allows for fast pre-compiled operations. It's a little more code and overhead to run but for large computations will be worth the effort. It also allows reading and processing data larger than the available RAM memory.

In [ ]:
import dask.array as da  # Convention is to import as da

Create a Dask data array with dask module, similar to Numpy. We define chunks to help manage multiprocessing and allow for smaller RAM memory footprint. We could process data larger than the RAM memory we have available to us. Jupyter and Dask play well together and Jupyter will print a nice graphical output of the Dask array size/shape/type. Notice when we print it does not print the values, only information about the Dask array. That is because the values have not been created yet, only the steps to perform the operation are created.

In [ ]:
x = da.random.random((10000, 10000), chunks=(1000, 1000))
x

Perform math similar to Numpy.

In [ ]:
y = x + x.T  # .T means transpose the array

Here we calculate the mean along an axis.

But notice when we print z it does not show any values, just information about the dask object. At this time no computation has occured, just the information about what we want to do.

In [ ]:
z = y[::2, 5000:].mean(axis=1)
z

We will need to computer the values and convert to Numpy before printing the values to the screen. .compute() on the object will return a Numpy array of the values.

In [ ]:
z = z.compute()
z

Once we call .computer() Jupyter will treat z as a normal Numpy array. We could have also delayed all the processing and rolled the greater than and .any() computation into Dask for faster overall computation.

In [ ]:
a = z > 0  # Return array of boolean values where value greater than zero
b = a.any()  # Are any values set to True
b

Can also perform all computations delayed.

In [ ]:
x = da.random.random((10000, 10000), chunks=(1000, 1000))
y = x + x.T  # .T means transpose the array
z = y[::2, 5000:].mean(axis=1)
a = z > 0  # Return array of boolean values where value greater than zero
b = a.any()  # Are any values set to True
print('b:', b)
b = b.compute()
print('b:', b)

Most of the Numpy operations are available in Dask computations. Notice that the nanmean() method is not actually run yet. The printed variable "c" is a placeholder for the operation.

In [ ]:
a = da.random.random(1000, chunks=100)
b = da.ones(1000, chunks=100)

c = b - a
c = da.nanmean(c)
c

We need to tell Dask to perform the computation to produce the value.

In [ ]:
c = c.compute()
c

## How much faster is dask than Numpy at some calculations?

Let's make a large array.

We will import a different library to do the timing to demonstrate the difference in computation speeds.

In [ ]:
import time
import numpy as np

Let's create two functions that perform the same task. One uses Dask and one uses Numpy. They return the number of seconds to make the computations.

In [ ]:
def numpy_computation(num):
    start_time = time.time()
    b = np.ones(num) - np.random.random(num)
    b[np.random.randint(0, num, int(num/10))] = np.nan
    b = np.nanmean(b)

    return time.time() - start_time
    
def dask_computation(num):
    start_time = time.time()
    chunks = int(num/10)
    b = da.ones(num, chunks=chunks) - da.random.random(num, chunks=chunks)
    b[da.random.randint(0, 1, num, chunks=chunks).astype(bool)] = np.nan
    b = da.nanmean(b)
    b = b.compute()

    return time.time() - start_time

In [ ]:
num = 100_000_000
numpy_time = numpy_computation(num)
dask_time = dask_computation(num)

print(f'Numpy Elapsed Time: {numpy_time} seconds')
print(f'Dask Elapsed Time: {dask_time} seconds')

print(f'\nDask is about {numpy_time/dask_time} times faster than Numpy for this operation.\n')

What about a smaller array. It takes overhead operations to perform the work so for small operations Dask can be slower than normal Numpy operations.

In [ ]:
num = 1000
numpy_time = numpy_computation(num)
dask_time = dask_computation(num)

print(f'Numpy Elapsed Time: {numpy_time} seconds\n')
print(f'Dask Elapsed Time: {dask_time} seconds\n')

print(f'\nNumpy is about {dask_time/numpy_time} times faster than Dask for this much smaller operation.\n')